In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import datetime as dt
import statsmodels.api as sm

In [2]:
end = dt.date(2021, 6, 30)
start = dt.date(2016, 7, 5)
index = ['^GSPC']

In [39]:
interval = '1d'
period1 = int(time.mktime(dt.datetime(2016, 7, 1, 23, 59).timetuple()))
period2 = int(time.mktime(dt.datetime(2021, 7, 1, 23, 59).timetuple()))
index_ret = pd.DataFrame()

xlwriter = pd.ExcelWriter('historical prices.xlsx', engine='openpyxl')
for ticker in index:
    try:
        query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
        df = pd.read_csv(query_string)
        df.drop(columns=['Open', 'Close', 'Low', 'High', 'Volume'], inplace=True)
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        df.rename(columns={'Adj Close': ticker}, inplace=True)
        index_ret = pd.concat([index_ret, df], axis=1)
    except:
        print('No data for: ', ticker)

In [40]:
index_ret = index_ret.pct_change()

In [41]:
index_ret = index_ret[1:]

In [43]:
index_ret.to_csv('sp500_returns.csv')

In [64]:
all_factors = pd.read_csv('all_factors_frame.csv')
all_factors.set_index('Date', inplace=True)

In [65]:
index_ret = index_ret[:-1]

In [66]:
index_ret.head(3)

,^GSPC
Date,
2016-07-05,-0.006847
2016-07-06,0.005353
2016-07-07,-0.000872


In [67]:
all_factors.rename(columns={'^GSPC':'SP500'}, inplace=True)

In [68]:
all_factors[['Mkt-RF', 'SMB_Final', 'HML', 'RF']] = all_factors[['Mkt-RF', 'SMB_Final', 'HML', 'RF']]/100

In [69]:
all_factors['SP500-RF'] = all_factors['SP500'] - all_factors['RF']

In [83]:
all_factors

,Mkt-RF,SMB_Final,HML,BRF,RF,SP500,SMB,SP500-RF
Date,,,,,,,,
2016-07-05,-0.0085,-0.003073,-0.0137,-0.010961,0.00001,-0.006847,-0.61,-0.006857
2016-07-06,0.0062,0.000861,-0.0014,0.005503,0.00001,0.005353,0.17,0.005343
2016-07-07,0.0002,0.001703,-0.0019,0.002984,0.00001,-0.000872,0.34,-0.000882
2016-07-08,0.0160,0.004487,0.0033,0.017619,0.00001,0.015253,0.89,0.015243
2016-07-11,0.0043,0.002984,0.0026,0.005230,0.00001,0.003409,0.59,0.003399
...,...,...,...,...,...,...,...,...
2021-06-24,0.0068,-0.000512,0.0023,0.006636,0.00000,0.005811,0.62,0.005811
2021-06-25,0.0032,-0.000579,0.0046,0.006592,0.00000,0.003331,-0.68,0.003331
2021-06-28,0.0023,-0.000542,-0.0159,-0.003669,0.00000,0.002315,-0.48,0.002315


In [84]:
y = all_factors['SP500-RF']
x = all_factors[['Mkt-RF', 'SMB', 'HML']]

X_sm = sm.add_constant(x)

In [85]:
model = sm.OLS(y,X_sm)
results = model.fit()
result_csv = results.summary()
result_csv

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               SP500-RF   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 1.457e+05
Date:                Thu, 02 Sep 2021   Prob (F-statistic):               0.00
Time:                        18:13:38   Log-Likelihood:                 7457.8
No. Observations:                1257   AIC:                        -1.491e+04
Df Residuals:                    1253   BIC:                        -1.489e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -8.919e-05   1.82e-05     -4.912      0.000      -0.000   -5.36e-05
Mkt-RF         0.9902      0.002    655.169      0.000       0.987       0.993
SMB           -0.0014   2.85e-05    -47.428      0.000      -0.001      -0.001
HML            0.0241      0.002     12.468      0.000       0.020       0.028
==============================================================================
Omnibus:                       73.711   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              268.165
Skew:                           0.134   Prob(JB):                     5.87e-59
Kurtosis:                       5.247   Cond. No.                         107.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Mkt-RF = 0.9902, less market risk than the average asset

SMB = -0.0014, negative so invests very slightly weighted into big stocks

HML = 0.0241, less growth stocks

const = alpha = -0.0002, tiny negative so negative performance of the risk of the three fama french factors

In [63]:
""" ADJUST FOR DIFFERENT SMB VALUES!!!!!!! """

' ADJUST FOR DIFFERENT SMB VALUES!!!!!!! '

In [73]:
y = all_factors['SP500-RF']
x = all_factors[['Mkt-RF', 'SMB_Final', 'HML', 'BRF']]

X_sm = sm.add_constant(x)

In [74]:
model = sm.OLS(y,X_sm)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               SP500-RF   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                 4.305e+04
Date:                Thu, 02 Sep 2021   Prob (F-statistic):               0.00
Time:                        18:04:52   Log-Likelihood:                 6875.5
No. Observations:                1257   AIC:                        -1.374e+04
Df Residuals:                    1252   BIC:                        -1.372e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0002   3.03e-05     -6.756      0.000      -0.000      -0.000
Mkt-RF         0.9563      0.009    110.103      0.000       0.939       0.973
SMB_Final     -0.2546      0.023    -11.154      0.000      -0.299      -0.210
HML            0.0112      0.003      3.532      0.000       0.005       0.017
BRF            0.0272      0.008      3.215      0.001       0.011       0.044
==============================================================================
Omnibus:                      134.890   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1018.685
Skew:                          -0.125   Prob(JB):                    6.24e-222
Kurtosis:                       7.403   Cond. No.                         793.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [86]:
br_ticker = ['BIBL']

interval = '1d'
period1 = int(time.mktime(dt.datetime(2016, 7, 1, 23, 59).timetuple()))
period2 = int(time.mktime(dt.datetime(2021, 7, 1, 23, 59).timetuple()))
index_ret = pd.DataFrame()

xlwriter = pd.ExcelWriter('historical prices.xlsx', engine='openpyxl')
for ticker in br_ticker:
    try:
        query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
        df = pd.read_csv(query_string)
        df.drop(columns=['Open', 'Close', 'Low', 'High', 'Volume'], inplace=True)
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        df.rename(columns={'Adj Close': ticker}, inplace=True)
        index_ret = pd.concat([index_ret, df], axis=1)
    except:
        print('No data for: ', ticker)

In [131]:
br_returns = pd.read_csv('bibl_returns.csv')
br_returns.set_index('Date', inplace=True)
br_returns['BIBL'] = br_returns['BIBL'].pct_change()
br_returns = br_returns.iloc[1:-1]
br_returns

,BIBL
Date,
2017-11-01,-0.001196
2017-11-02,-0.001158
2017-11-03,0.002918
2017-11-06,0.006416
2017-11-07,0.000990
...,...
2021-06-24,0.006294
2021-06-25,0.003265
2021-06-28,0.005164


In [132]:
br_factors = pd.read_csv('all_factors_frame.csv')
br_factors.set_index(br_factors['Date'], inplace=True)
br_factors.drop(columns=['^GSPC'], inplace=True)
br_factors = br_factors.loc['2017-11-01':'2021-06-30']


In [133]:
br_factors

,Date,Mkt-RF,SMB_Final,HML,BRF,RF,SMB
Date,,,,,,,
2017-11-01,2017-11-01,0.05,-0.058576,0.11,-0.004959,0.004,-0.76
2017-11-02,2017-11-02,0.06,-0.052855,0.50,-0.006252,0.004,-0.04
2017-11-03,2017-11-03,0.31,-0.056410,-0.73,0.008972,0.004,-0.41
2017-11-06,2017-11-06,0.06,-0.055123,0.19,0.004370,0.004,0.23
2017-11-07,2017-11-07,-0.20,-0.060235,-0.40,-0.003730,0.004,-1.36
...,...,...,...,...,...,...,...
2021-06-24,2021-06-24,0.68,-0.051192,0.23,0.006636,0.000,0.62
2021-06-25,2021-06-25,0.32,-0.057910,0.46,0.006592,0.000,-0.68
2021-06-28,2021-06-28,0.23,-0.054243,-1.59,-0.003669,0.000,-0.48


In [134]:
full_br_frame = pd.concat([br_factors, br_returns], axis=1)

In [135]:
full_br_frame[['Mkt-RF', 'SMB_Final', 'HML', 'RF']] = full_br_frame[['Mkt-RF', 'SMB_Final', 'HML', 'RF']]/100

In [136]:
full_br_frame['BIBL-RF'] = full_br_frame['BIBL'] - full_br_frame['RF']

In [137]:
full_br_frame

,Date,Mkt-RF,SMB_Final,HML,BRF,RF,SMB,BIBL,BIBL-RF
Date,,,,,,,,,
2017-11-01,2017-11-01,0.0005,-0.000586,0.0011,-0.004959,0.00004,-0.76,-0.001196,-0.001236
2017-11-02,2017-11-02,0.0006,-0.000529,0.0050,-0.006252,0.00004,-0.04,-0.001158,-0.001198
2017-11-03,2017-11-03,0.0031,-0.000564,-0.0073,0.008972,0.00004,-0.41,0.002918,0.002878
2017-11-06,2017-11-06,0.0006,-0.000551,0.0019,0.004370,0.00004,0.23,0.006416,0.006376
2017-11-07,2017-11-07,-0.0020,-0.000602,-0.0040,-0.003730,0.00004,-1.36,0.000990,0.000950
...,...,...,...,...,...,...,...,...,...
2021-06-24,2021-06-24,0.0068,-0.000512,0.0023,0.006636,0.00000,0.62,0.006294,0.006294
2021-06-25,2021-06-25,0.0032,-0.000579,0.0046,0.006592,0.00000,-0.68,0.003265,0.003265
2021-06-28,2021-06-28,0.0023,-0.000542,-0.0159,-0.003669,0.00000,-0.48,0.005164,0.005164


In [138]:
y = full_br_frame['BIBL-RF']
x = full_br_frame[['Mkt-RF', 'SMB', 'HML']]

X_sm = sm.add_constant(x)

In [139]:
model = sm.OLS(y,X_sm)
results = model.fit()
result_csv = results.summary()
result_csv

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                BIBL-RF   R-squared:                       0.922
Model:                            OLS   Adj. R-squared:                  0.922
Method:                 Least Squares   F-statistic:                     3610.
Date:                Thu, 02 Sep 2021   Prob (F-statistic):               0.00
Time:                        18:54:09   Log-Likelihood:                 3775.6
No. Observations:                 921   AIC:                            -7543.
Df Residuals:                     917   BIC:                            -7524.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.397e-05      0.000      0.180      0.857      -0.000       0.000
Mkt-RF         0.9855      0.010    101.918      0.000       0.966       1.004
SMB         3.015e-06      0.000      0.016      0.988      -0.000       0.000
HML            0.0833      0.013      6.590      0.000       0.058       0.108
==============================================================================
Omnibus:                      119.801   Durbin-Watson:                   2.310
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              760.974
Skew:                          -0.386   Prob(JB):                    5.71e-166
Kurtosis:                       7.385   Cond. No.                         95.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [140]:
y = full_br_frame['BIBL-RF']
x = full_br_frame[['Mkt-RF', 'SMB_Final', 'HML', 'BRF']]

X_sm = sm.add_constant(x)

In [141]:
model = sm.OLS(y,X_sm)
results = model.fit()
result_csv = results.summary()
result_csv

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                BIBL-RF   R-squared:                       0.927
Model:                            OLS   Adj. R-squared:                  0.927
Method:                 Least Squares   F-statistic:                     2908.
Date:                Thu, 02 Sep 2021   Prob (F-statistic):               0.00
Time:                        18:58:29   Log-Likelihood:                 3806.5
No. Observations:                 921   AIC:                            -7603.
Df Residuals:                     916   BIC:                            -7579.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0017      0.002     -0.842      0.400      -0.006       0.002
Mkt-RF         0.7149      0.037     19.504      0.000       0.643       0.787
SMB_Final     -2.9448      3.618     -0.814      0.416     -10.045       4.155
HML            0.0528      0.014      3.768      0.000       0.025       0.080
BRF            0.2772      0.036      7.779      0.000       0.207       0.347
==============================================================================
Omnibus:                      118.600   Durbin-Watson:                   2.445
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              935.018
Skew:                          -0.275   Prob(JB):                    9.19e-204
Kurtosis:                       7.905   Cond. No.                     2.82e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.82e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""